In [1]:
"""
from google.colab import drive
drive.mount('/content/drive')
"""

Mounted at /content/drive


In [2]:
working_directory = '../'

In [3]:
!pip install transformers

     |████████████████████████████████| 4.0 MB 30.9 MB/s 
     |████████████████████████████████| 77 kB 8.2 MB/s 
     |████████████████████████████████| 880 kB 61.3 MB/s 
     |████████████████████████████████| 6.6 MB 60.4 MB/s 
     |████████████████████████████████| 596 kB 58.4 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.51-py3-none-any.whl size=895170 sha256=21473355083ecaaf1eb6fb644f0fadc35b28e1736999a2a2d3f8fe6f886e6596
  Stored in directory: /root/.cache/pip/wheels/a0/95/bf/f9b021d14dbea2160ac68cbdf606e42e01119e0ba7206a6157
Successfully built sacremoses
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [4]:
import torch

if torch.cuda.is_available():       
    device = torch.device("cuda")
    print('Using GPU.')
else:
    device = torch.device("cpu")
    print('Using CPU.')


Using GPU.


#**Data Prep**

In [5]:
from transformers import RobertaTokenizer

rob_tokenizer = RobertaTokenizer.from_pretrained('roberta-base', do_lower_case=True)

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

In [6]:
import os

eng_path = working_directory + 'data/2017_English_final/GOLD/Subtask_A/'
train_files = []
val_files = []

for root, dirs, files in os.walk(eng_path):
    for file_name in files:
        if 'train' in file_name and '.txt' in file_name:
            train_files.append(os.path.join(eng_path, file_name))
        if 'dev' in file_name and '.txt' in file_name:
            val_files.append(os.path.join(eng_path, file_name))
print(train_files)
print(val_files)


['drive/MyDrive/CS505/Project/data/2017_English_final/GOLD/Subtask_A/twitter-2013train-A.txt', 'drive/MyDrive/CS505/Project/data/2017_English_final/GOLD/Subtask_A/twitter-2015train-A.txt', 'drive/MyDrive/CS505/Project/data/2017_English_final/GOLD/Subtask_A/twitter-2016train-A.txt']
['drive/MyDrive/CS505/Project/data/2017_English_final/GOLD/Subtask_A/twitter-2016dev-A.txt', 'drive/MyDrive/CS505/Project/data/2017_English_final/GOLD/Subtask_A/twitter-2013dev-A.txt', 'drive/MyDrive/CS505/Project/data/2017_English_final/GOLD/Subtask_A/twitter-2016devtest-A.txt']


In [7]:
import numpy as np


train_data = []
train_labels = []
val_data = []
val_labels = []

sentiment_to_label = {'positive': 2, 'neutral': 1, 'negative': 0}

for file_path in train_files:
    with open(file_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()
        for l in lines:
            entries = l.split('\t')
            train_data.append(entries[2])
            train_labels.append(sentiment_to_label[entries[1]])
            
    
for file_path in val_files:
    with open(file_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()
        for l in lines:
            entries = l.split('\t')
            val_data.append(entries[2])
            val_labels.append(sentiment_to_label[entries[1]])

train_data = np.array(train_data)
train_labels = np.array(train_labels)
val_data = np.array(val_data)
val_labels = np.array(val_labels)

In [8]:
max_len_train = 0

# For every sentence...
for sent in train_data:

    input_ids = rob_tokenizer.encode(sent, add_special_tokens=True)

    # Update the maximum sentence length.
    max_len_train = max(max_len_train, len(input_ids))

print('Max sentence length: ', max_len_train)

Max sentence length:  94


In [9]:
max_len_val = 0

# For every sentence...
for sent in val_data:

    input_ids = rob_tokenizer.encode(sent, add_special_tokens=True)

    # Update the maximum sentence length.
    max_len_val = max(max_len_val, len(input_ids))

print('Max sentence length: ', max_len_val)

Max sentence length:  114


In [10]:
max_len = max(max_len_train, max_len_val)

def tokenize(data,labels, tokenizer):
  input_ids=[]
  attention_masks = []
  # For every sentence...
  for sent in data:
    encoded_dict = tokenizer.encode_plus(
                        sent,
                        add_special_tokens = True, 
                        max_length = max_len+100,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

  # Convert the lists into tensors.
  input_ids = torch.cat(input_ids, dim=0)
  attention_masks = torch.cat(attention_masks, dim=0)
  label = torch.tensor(labels)
  return input_ids,attention_masks,label

In [11]:
train_input_ids, train_attention_masks,train_label=tokenize(train_data,train_labels, rob_tokenizer)


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2269: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [12]:
val_input_ids, val_attention_masks, val_label=tokenize(val_data,val_labels, rob_tokenizer)


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2269: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [13]:
print('Original: ', train_data[0])
print('Token IDs:', train_input_ids[0])
print(rob_tokenizer.decode(train_input_ids[0]))

Original:  Gas by my house hit $3.39!!!! I\u2019m going to Chapel Hill on Sat. :)

Token IDs: tensor([    0, 34974,    30,   127,   790,   478,    68,   246,     4,  3416,
        32376,    38, 37457,   257, 10626,   119,   164,     7, 12683,  1934,
           15,  8918,     4, 44660, 50118,     2,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1

In [14]:
from torch.utils.data import TensorDataset
train_dataset = TensorDataset(train_input_ids, train_attention_masks, train_label)
val_dataset = TensorDataset(val_input_ids, val_attention_masks, val_label)

In [15]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
batch_size = 16

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order. 
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
validation_dataloader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

#**Loading Model**

In [17]:
from transformers import RobertaForSequenceClassification
model = RobertaForSequenceClassification.from_pretrained("roberta-base",
                                             num_labels = 3,
                                             output_attentions = False,
                                             output_hidden_states = False, 
)

model.to(device)

Downloading:   0%|          | 0.00/478M [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifi

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerN

#**Fine-Tuning**

In [18]:
from transformers import AdamW
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [19]:
from transformers import get_linear_schedule_with_warmup

# Number of training epochs. The BERT authors recommend between 2 and 4. 
# We chose to run for 4, but we'll see later that this may be over-fitting the
# training data.
epochs = 4

# Total number of training steps is [number of batches] x [number of epochs]. 
# (Note that this is not the same as the number of training samples).
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

In [20]:
import numpy as np

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [21]:
import random

def train(model, train_dataloader, val_dataloader, epochs, print_every=40, seed_val=42):
  random.seed(42)
  np.random.seed(seed_val)
  torch.manual_seed(seed_val)
  torch.cuda.manual_seed_all(seed_val)

  training_stats = []

  total_t0 = time.time()

  for epoch_i in range(epochs):
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_train_loss = 0

    model.train()

    for step, batch in enumerate(train_dataloader):
      if step % print_every == 0 and not step == 0:

        elapsed = format_time(time.time() - t0)

        print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

      b_input_ids = batch[0].to(device)
      b_input_mask = batch[1].to(device)
      b_labels = batch[2].to(device)

      model.zero_grad()

      result = model(b_input_ids, 
                     token_type_ids=None, 
                     attention_mask=b_input_mask, 
                     labels=b_labels,
                     return_dict=True)
      
      loss = result.loss
      logits = result.logits

      total_train_loss += loss.item()
      loss.backward()

      torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

      optimizer.step()
      scheduler.step()

    avg_train_loss = total_train_loss / len(train_dataloader)            

    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))


    print("")
    print("Running Validation...")

    t0 = time.time()

    model.eval()

    # Tracking variables 
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    for batch in validation_dataloader:
      
      b_input_ids = batch[0].to(device)
      b_input_mask = batch[1].to(device)
      b_labels = batch[2].to(device)

      with torch.no_grad():
        result = model(b_input_ids, 
                       token_type_ids=None, 
                       attention_mask=b_input_mask,
                       labels=b_labels,
                       return_dict=True)
        
      loss = result.loss
      logits = result.logits

      total_eval_loss += loss.item()

      logits = logits.detach().cpu().numpy()
      label_ids = b_labels.to('cpu').numpy()

      total_eval_accuracy += flat_accuracy(logits, label_ids)
    
    # Report the final accuracy for this validation run.
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(validation_dataloader)
    
    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)
    
    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

  print("")
  print("Training complete!")

  print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))

In [22]:
train(model, train_dataloader, validation_dataloader, epochs)


======== Epoch 1 / 4 ========
Training...
  Batch    40  of  1,011.    Elapsed: 0:00:22.
  Batch    80  of  1,011.    Elapsed: 0:00:44.
  Batch   120  of  1,011.    Elapsed: 0:01:07.
  Batch   160  of  1,011.    Elapsed: 0:01:30.
  Batch   200  of  1,011.    Elapsed: 0:01:54.
  Batch   240  of  1,011.    Elapsed: 0:02:17.
  Batch   280  of  1,011.    Elapsed: 0:02:41.
  Batch   320  of  1,011.    Elapsed: 0:03:04.
  Batch   360  of  1,011.    Elapsed: 0:03:28.
  Batch   400  of  1,011.    Elapsed: 0:03:51.
  Batch   440  of  1,011.    Elapsed: 0:04:15.
  Batch   480  of  1,011.    Elapsed: 0:04:38.
  Batch   520  of  1,011.    Elapsed: 0:05:02.
  Batch   560  of  1,011.    Elapsed: 0:05:25.
  Batch   600  of  1,011.    Elapsed: 0:05:49.
  Batch   640  of  1,011.    Elapsed: 0:06:12.
  Batch   680  of  1,011.    Elapsed: 0:06:36.
  Batch   720  of  1,011.    Elapsed: 0:07:00.
  Batch   760  of  1,011.    Elapsed: 0:07:23.
  Batch   800  of  1,011.    Elapsed: 0:07:47.
  Batch   840  of

In [23]:
torch.save(model, working_directory + 'Roberta/fine_tuned_roberta.pt')

#**Testing**

In [ ]:
model = torch.load(working_directory + 'Roberta/fine_tuned_roberta.pt')
model.to(device)

In [24]:
test_path = working_directory + 'data/2017_Arabic_train_final/GOLD/SemEval2017-task4-train.subtask-A.english.txt'

test_data = []
test_labels = []

with open(test_path, 'r', encoding='utf-8') as f:
    lines = f.readlines()
    for l in lines:
        entries = l.split('\t')
        if len(entries) != 3:
            entries = l.split(' ', maxsplit=2)
        test_data.append(entries[2])
        test_labels.append(sentiment_to_label[entries[1]])

In [25]:
test_data = np.array(test_data)
test_labels = np.array(test_labels)

In [27]:
input_ids_test, attention_masks_test, labels_test = tokenize(test_data, test_labels, rob_tokenizer)

# Create the DataLoader.
prediction_data = TensorDataset(input_ids_test, attention_masks_test, labels_test)
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2269: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [28]:
print('Predicting labels for {:,} test sentences...'.format(len(input_ids_test)))

# Put model in evaluation mode
model.eval()

# Tracking variables 
predictions , true_labels = [], []

# Predict 
for batch in prediction_dataloader:
  # Add batch to GPU
  batch = tuple(t.to(device) for t in batch)
  
  # Unpack the inputs from our dataloader
  b_input_ids, b_input_mask, b_labels = batch
  
  # Telling the model not to compute or store gradients, saving memory and 
  # speeding up prediction
  with torch.no_grad():
      # Forward pass, calculate logit predictions.
      result = model(b_input_ids, 
                     token_type_ids=None, 
                     attention_mask=b_input_mask,
                     return_dict=True)

  logits = result.logits

  # Move logits and labels to CPU
  logits = logits.detach().cpu().numpy()
  label_ids = b_labels.to('cpu').numpy()
  
  # Store predictions and true labels
  predictions.append(logits)
  true_labels.append(label_ids)

Predicting labels for 3,355 test sentences...


In [29]:
predictions = [p for sublist in predictions for p in sublist]
true_labels = np.array([l for sublist in true_labels for l in sublist])
print(flat_accuracy(predictions, true_labels))

0.612220566318927
